# Stage 10a – Linear Regression Project

Baseline regression model on engineered features to predict daily returns.

In [ ]:
import pandas as pd
from pathlib import Path

# Load dataset with engineered features (Stage 9)
data_path = Path("../data/processed/IYR_features_project.csv")
df = pd.read_csv(data_path)
df.head()